# Linear Elasticity

*Authors:* 
 - *Jack Hale, (Univ. Luxembourg)*
 - *Laura De Lorenzis (ETH Zürich)*
 - *Corrado Maurini (corrado.maurini@sorbonne-universite.fr)*

This notebook serves as a tutorial to solve a problem of linear elasticity using DOLFINx (FEniCS).

You can find a tutorial and useful resources for DOLFINx at the following links

- https://docs.fenicsproject.org/
- https://github.com/FEniCS/dolfinx/
- https://jorgensd.github.io/dolfinx-tutorial/, see https://jorgensd.github.io/dolfinx-tutorial/chapter2/linearelasticity.html for linear elasticity

We consider an elastic slab $\Omega$ with a straight crack $\Gamma$ subject to a mode-I loading by an applied traction force $f$, see figure. 

Using the symmetry, we will consider only half of the domain in the computation.

![title](./figures/domain.png)

We solve the problem of linear elasticity with the finite element method, implemented using DOLFINx.

DOLFINx is advanced library that allows for efficient parallel computation. For the sake of simplicity, we assume here to work on a single processor and will not use MPI-related commands. Using DOLFINx with MPI will be covered in the afternoon session.

We start importing the required libraries. 

In [1]:
# Import required libraries
import matplotlib.pyplot as plt
import numpy as np

import dolfinx.fem as fem
import dolfinx.mesh as mesh
import dolfinx.io as io
import dolfinx.plot as plot
import ufl

from mpi4py import MPI
from petsc4py import PETSc
from petsc4py.PETSc import ScalarType

Let us generate a mesh using gmsh (http://gmsh.info/). 
The mesh is refined around the crack tip.
The function to generate the mesh is reported in the external file `meshes.py` located in the directory `python`.
To import it, we add `python` to the path where the system is looking for functions to import

In [ ]:
import sys
sys.path.append("python")
from meshes import generate_mesh_with_crack

In [2]:
Lx = 1.
Ly = 0.5
Lcrack = 0.3
lc =.2
dist_min = .1
dist_max = .3
msh, mt, ft = generate_mesh_with_crack(
        Lcrack=Lcrack,
        Ly=0.5,
        lc=0.1,  # caracteristic length of the mesh
        refinement_ratio=10,  # how much it is refined at the tip zone
        dist_min=dist_min,  # radius of tip zone
        dist_max=dist_max,  # radius of the transition zone
    )

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 20%] Meshing curve 2 (Line)
Info    : [ 40%] Meshing curve 3 (Line)
Info    : [ 60%] Meshing curve 4 (Line)
Info    : [ 80%] Meshing curve 5 (Line)
Info    : Done meshing 1D (Wall 0.00214083s, CPU 0.00223s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.00644637s, CPU 0.006358s)
Info    : 413 nodes 829 elements


To plot the mesh we use `pyvista` see:
- https://jorgensd.github.io/dolfinx-tutorial/chapter3/component_bc.html
- https://docs.fenicsproject.org/dolfinx/main/python/demos/pyvista/demo_pyvista.py.html

In [3]:
import pyvista
pyvista.set_jupyter_backend("pythreejs")
grid = pyvista.UnstructuredGrid(*plot.create_vtk_mesh(msh))
plotter = pyvista.Plotter()
plotter.add_mesh(grid, show_edges=True)
plotter.camera_position = 'xy'
pyvista.OFF_SCREEN = False
if not pyvista.OFF_SCREEN:
    plotter.show()

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(intensity=0.25, positi…

## Finite element function space

We use here linear Lagrange triangle elements

In [4]:
element = ufl.VectorElement('Lagrange',msh.ufl_cell(),degree=1,dim=2)
V = fem.FunctionSpace(msh, element)

## Dirichlet boundary conditions

We define below the functions to impose the Dirichlet boundary conditions.

In our case we want to 
- block the vertical component $u_1$ of the displacement on the part of the bottom boundary without crack
- block the horizontal component $u_0$ on the right boundary

We first get the facets to block on the boundary (`dolfinx.mesh.locate_entities_boundary`) and then the corresponding dofs (`dolfinx.fem.locate_dofs_topological`)

In [5]:
def bottom_no_crack(x):
    return np.logical_and(np.isclose(x[1], 0.0), 
                          x[0] > Lcrack)

def right(x):
    return np.isclose(x[0], Lx)

In [6]:
bottom_no_crack_facets = mesh.locate_entities_boundary(msh, msh.topology.dim-1, bottom_no_crack)
bottom_no_crack_dofs_y = fem.locate_dofs_topological(V.sub(1), msh.topology.dim-1, bottom_no_crack_facets)

right_facets = mesh.locate_entities_boundary(msh, msh.topology.dim-1, right)
right_dofs_x = fem.locate_dofs_topological(V.sub(0), msh.topology.dim-1, right_facets)

The following lines define the `dolfinx.fem.dirichletbc` objects. We impose a zero displacement.

In [7]:
bc_bottom = fem.dirichletbc(ScalarType(0), bottom_no_crack_dofs_y, V.sub(1))
bc_right = fem.dirichletbc(ScalarType(0), right_dofs_x, V.sub(0))
bcs = [bc_bottom, bc_right]

## Define the bulk and surface mesures
The bulk (`dx`) and surface (`ds`) measures are used by `ufl` to write variational form with integral over the domain or the boundary, respectively. 

In this example the surface measure `ds` includes tags to specify Neumann bcs: `ds(1)` will mean the integral on the top boundary. 

In [8]:
dx = ufl.Measure("dx",domain=msh)
top_facets = mesh.locate_entities_boundary(msh, 1, lambda x : np.isclose(x[1], Ly))
mt = mesh.meshtags(msh, 1, top_facets, 1)
ds = ufl.Measure("ds", subdomain_data=mt)

In Python, you can get help on the different functions with the folling syntax:

In [9]:
help(mesh.meshtags)

Help on function meshtags in module dolfinx.mesh:

meshtags(mesh: 'Mesh', dim: 'int', indices: 'np.ndarray', values: 'typing.Union[np.ndarray, int, float]') -> 'MeshTagsMetaClass'
    Create a MeshTags object that associates data with a subset of mesh entities.
    
    Args:
        mesh: The mesh
        dim: Topological dimension of the mesh entity
        indices: Entity indices (local to process)
        values: The corresponding value for each entity
    
    Returns:
        A MeshTags object
    
    Note:
        The type of the returned MeshTags is inferred from the type of
        ``values``.



## Define the variational problem 

We specify the problem to solve though the weak formulation written in the [ufl](https://fenics.readthedocs.io/projects/ufl/en/latest/) syntax by giving the bilinear $a(u,v)$ and linear forms $L(v)$ in 
the weak formulation: 

find the *trial function* $u$ such that for all *test function* $v$
$a(u,v)=L(v)$ with 

$$
a(u,v)=\int_{\Omega\setminus\Gamma}\sigma(\varepsilon(u))\cdot \varepsilon(v)\,\mathrm{d}x, 
\quad L(v)=\int_\Omega b\cdot v \,\mathrm{d}x + \int_{\partial_N\Omega} f\cdot v \,\mathrm{d}s 
$$

Note on UFL terminology:
- `ufl.inner(sigma(eps(u)), eps(v))` is an expression
- `ufl.inner(sigma(eps(u)), eps(v)) * dx` is a form

In [10]:
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

E = 1. 
nu = 0.3 
mu = E / (2.0 * (1.0 + nu))
lmbda = E * nu / ((1.0 + nu) * (1.0 - 2.0 * nu))
# this is for plane-stress
lmbda = 2 * mu * lmbda / ( lmbda + 2 * mu )

def eps(u):
    """Strain"""
    return ufl.sym(ufl.grad(u))

def sigma(eps):
    """Stress"""
    return 2.0 * mu * eps + lmbda * ufl.tr(eps) * ufl.Identity(2)

def a(u,v):
    """The bilinear form of the weak formulation"""
    return ufl.inner(sigma(eps(u)), eps(v)) * dx 

def L(v): 
    """The linear form of the weak formulation"""
    # Volume force
    b = fem.Constant(msh,ScalarType((0, 0)))

    # Surface force on the top
    f = fem.Constant(msh,ScalarType((0, 0.1)))
    return ufl.dot(b, v) * dx + ufl.dot(f, v) * ds(1)

Let us plot the solution using `pyvista`, see
- https://jorgensd.github.io/dolfinx-tutorial/chapter3/component_bc.html
- https://docs.fenicsproject.org/dolfinx/v0.5.0/python/demos/demo_pyvista.html

## Define the linear problem and solve
We solve the problem using a direct solver. The class `dolfinx.fem.LinearProblem` assemble the stiffness matrix and load vector, apply the boundary conditions, and solve the linear system.

In [11]:
problem = fem.petsc.LinearProblem(a(u,v), L(v), bcs=bcs, 
                                    petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
uh = problem.solve()
uh.name = "displacement"

## Postprocessing

We can easily calculate the potential energy

In [12]:
energy = fem.assemble_scalar(fem.form(0.5 * a(uh, uh) - L(uh)))
print(f"The potential energy is {energy:2.3e}")

The potential energy is -4.165e-03


We can save the results to a file, that we can open with `paraview` (https://www.paraview.org/)

In [13]:
with io.XDMFFile(MPI.COMM_WORLD, "output/elasticity-demo.xdmf", "w") as file:
    file.write_mesh(uh.function_space.mesh)
    file.write_function(uh)

## Stress computation

We calculate here the Von Mises stress by interpolating the corresponding ufl expression, see https://jorgensd.github.io/dolfinx-tutorial/chapter2/linearelasticity_code.html#stress-computation

In [14]:
sigma_iso = 1./3*ufl.tr(sigma(eps(uh)))*ufl.Identity(uh.geometric_dimension())
sigma_dev =  sigma(eps(uh)) - sigma_iso
von_Mises = ufl.sqrt(3./2*ufl.inner(sigma_dev, sigma_dev))
V_von_mises = fem.FunctionSpace(msh, ("DG", 0))
stress_expr = fem.Expression(von_Mises, V_von_mises.element.interpolation_points())
vm_stress = fem.Function(V_von_mises)
vm_stress.interpolate(stress_expr)

In [15]:
from utils import warp_plot_2d
pyvista.set_jupyter_backend("pythreejs")
plotter = warp_plot_2d(uh,cell_field=vm_stress,field_name="Von Mises stress", factor=.5,show_edges=True,clim=[0, 0.3])
plotter.show()

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(intensity=0.25, positi…

We can now wrap all the code in a the external module, so that we can resuse the solver later

We define in `elastic_solver.py` a function `solve_elasticity` taking as input  the crack length `Lcrack`, the geoemtric and mesh parameters, the Poisson ratio `nu`, and giving us as output the solution field `uh` and the related potential energy `energy`

The returned `uh` and `energy` will be calculated assuming a force density `f=1` on the top surface and a Young modulus `E=1`. This is without loss of generality, see the exercise below.


**Exercise.** 
Let be $u^{*}$ and $P^{*}$ the displacement field obtained on a domain $\Omega^*=[0,L^*]\times[0,\varrho\, L^*]$ for a Young module $E^*$ and a load $f^*$ applied on the top surface. 
Determine by dimensional analysis the analytical formulas giving the 
displacement $u$ and the potential energy $P$ for any other value of  $E$,  load $f$, and for any domain $\Omega=[0,L]\times[0,\varrho\, L]$  obtained by a rescaling of $\Omega^*$ with a length-scale $L$
Deduce that we can, without loss of generality, perform computation with $E=1$, $f=1$ and $L=1$.

In [16]:
from mpi4py import MPI
from elastic_solver import solve_elasticity

uh, energy, _ = solve_elasticity(
    nu=0.3,
    E=1,
    load=1,
    Lx=1,
    Ly=0.5,
    Lcrack=0.3,
    lc=0.1,
    refinement_ratio=10,
    dist_min=0.2,
    dist_max=0.3,
    verbosity=1
)

with io.XDMFFile(MPI.COMM_WORLD, "output/elasticity-demo.xdmf", "w") as file:
    file.write_mesh(uh.function_space.mesh)
    file.write_function(uh)


The potential energy for Lcrack=3.000e-01 is -4.174e-01
